## protein sequence identity across genome assemblies

* find all assemblies for species we need
* download protein fasta
* make blast db for all seqs
* blast a tempalte target protein e.g. pknh1 
* find best hits for each species?
* store each protein seq per species and get identities etc.

alternatives could be to use roary for clustering?

In [31]:
import os, glob, subprocess, gzip, urllib
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import Phylo, AlignIO
from pylab import plt
import pandas as pd
import numpy as np
from epitopepredict import sequtils, utilities
pd.set_option('display.width', 180)
pd.set_option('max_colwidth', 120)
import tools

In [2]:
species = ['africanum','h37rv','h37ra','beijing','caprae','microti','ulcerans','marinum','canettii','BCG','bovis','pinnipedii','cdc']
def get_species(x):
    for s in species:
        if x.lower().find(s) != -1:
            return s
    return 'mtb'

In [85]:
df=pd.read_csv('genomes_data.csv')
asm = pd.read_csv('mtb_assemblies.csv')

filtered=asm[(asm.Level=='Complete Genome') | (~asm.species.isin(['mtb']))]
omit = ['PRJNA407834','PRJNA287158','PRJNA214551']
filtered = filtered[~filtered.BioProject.isin(omit)]

#include some extra mtb strains
include = ['']
#filtered=filtered[(filtered.species!='mtb')]
print (len(filtered))
print (filtered.species.value_counts())

255
mtb           87
BCG           37
africanum     31
bovis         28
marinum       25
H37Rv          9
canettii       9
ulcerans       7
CDC            7
H37Ra          3
caprae         3
microti        3
pinnipedii     3
orygis         2
liflandii      1
Name: species, dtype: int64


## create protein fasta of all target assemblies

In [ ]:
def create_protein_fasta():
    path='prot_assemblies'
    new=open('myco_proteins.faa','w')
    for i,row in list(filtered.iterrows()):
        acc = row.Assembly
        url = row['GenBank FTP']        
        filename = os.path.join(path, acc+'.faa.gz')
        if not os.path.exists(filename):
            label = os.path.basename(url)            
            link = os.path.join(url,label+'_protein.faa.gz')
            print (link)
            try:
                urllib.request.urlretrieve(link, filename)
            except:
                continue
        print (row.Assembly, row.Strain, row.BioProject)
        with gzip.open(filename, "rt") as handle:
            seqs = list(SeqIO.parse(handle, "fasta"))
            #print (len(seqs))
            for s in seqs:
                s.description += '_'+r.Strain
            new=open('myco_proteins.faa','a')
            SeqIO.write(seqs,new,'fasta')
            
create_protein_fasta()

## make blast db 

In [88]:
cmd='makeblastdb -in myco_proteins.faa -dbtype prot -out myco_proteins'
temp=subprocess.check_output(cmd,shell=True)

In [89]:
def find_orthologs(query, label):
    """Find hits in a target db and save unique sequences"""
    
    bl = sequtils.blast_sequences('myco_proteins',[query],cpus=8,maxseqs=200)
    #print (bl)
    bl=bl[(bl.pident>50) & (bl.qcovs>70)]
    #get unique
    bl=bl.drop_duplicates('sseq')
    print (len(bl))

    bl['strain'] = bl.stitle.apply(lambda x: x.split('_')[1])
    bl['species'] = bl.stitle.apply(get_species)
    cols=['sseqid','strain','species','pident','stitle']
    #print (bl[cols][:10])
    print ('found %s unique hits' %len(bl))
    print (bl.groupby('species').agg({'strain':np.size}))
    
    #save hits
    seqs = [SeqRecord(Seq(r.sseq), id=r.sseqid+'_'+r.species, description=r.species) for i,r in bl.iterrows()]
    seqs.append(query)
    SeqIO.write(seqs,'%s_hits.faa' %label,'fasta')
    #align results
    #alncmd="/usr/bin/mafft --auto --clustalout --reorder pknh1_hits.faa > pknh1_hits.aln"
    alncmd="muscle -in {l}_hits.faa -out {l}_hits.aln".format(l=label)
    tmp=subprocess.check_output(alncmd, shell=True)
    
    #aln = AlignIO.read('%s_hits.aln' %label,'fasta')
    #dm,tree=get_tree(aln)
    #Phylo.write([tree], '%s.newick' %label, 'newick')
    return bl


## blast a query

In [90]:
name='pknh3-pro'
#name='pknh1'
myseq = SeqIO.read('%s.fa' %name,'fasta')
print (myseq)
bl = find_orthologs(myseq, label=name)

ID: pknH3-proregion
Name: pknH3-proregion
Description: pknH3-proregion WP_012395731.1 serine/threonine protein kinase [Mycobacterium marinum] pknH3
Number of features: 0
Seq('MPYADPASSGPLPQSAPTGQPGWAPASGPIPAAHQPAQVPQYYQSGSWASTPPG...QPV', SingleLetterAlphabet())
6
found 6 unique hits
           strain
species          
africanum       1
canettii        1
h37rv           1
marinum         2
ulcerans        1


In [203]:
#r=Phylo.parse('trees.xml', 'phyloxml')
#tree=r.next()
#f,tr=draw_tree(tree,root='AOZ42422.1_mtb', title='pknh1')


1